In [45]:
# 미리 다운로드 받아야 할 파일
# yolov3.weights, yolov3.cfg, coco.names

import cv2
import sys
import os.path
import numpy as np

# 모델 & 설정 파일
model = "../models/yolo/yolov3.weights" #다운로드 링크 : https://pjreddie.com/media/files/yolov3.weights
config = "../models/yolo/yolov3.cfg" #다운로드 링크 : https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg
class_labels = "../models/yolo/coco.names" #다운로드 링크 : https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names

files = [model, config, class_labels]

#설정파일 있는지 체크
for f in files:
    if os.path.isfile(f) is False:
        print("모델 혹은 설정파일이 없습니다 : "+f)
        sys.exit()

confThreshold = 0.5
nmsThreshold = 0.4

# 테스트 이미지 파일
img_files = ["../images/dog1.jpg", "../images/person.jpg", "../images/sheep.jpg"]  #이미지 파일을 list로 저장

# 네트워크 생성
net = cv2.dnn.readNet(model, config) #모델 불러오기
if net.empty():
    print("Net open failed!")
    sys.exit()

# 클래스 이름 불러오기
classes = []
with open(class_labels, "rt") as f:
    classes = f.read().rstrip("\n").split("\n")

colors = np.random.uniform(0, 255, size=(len(classes), 3)) #80개의 클래스에 랜덤한 바운딩 박스 컬러 부여

# 출력 레이어 이름 받아오기, 82번, 94번, 106번 layer 사용
layer_names = net.getLayerNames()
output_layers = net.getUnconnectedOutLayersNames()
# output_layers = ["yolo_82", "yolo_94", "yolo_106"]

# 실행
for f in img_files:              #이미지 파일을 하나씩 읽어옴
    img = cv2.imread(f)

    if img is None:
        continue
        
    # 블롭 생성 & 추론
    blob = cv2.dnn.blobFromImage(img, 1/255., (320, 320), swapRB=True) #이미지 크기, 컬러 평균값 지정
    net.setInput(blob)  #입력설정
    outs = net.forward(output_layers)  #추론

    # outs는 3개의 ndarray 리스트.
    # outs[0].shape=(507, 85), 13*13*3=507
    # outs[1].shape=(2028, 85), 26*26*3=2028
    # outs[2].shape=(8112, 85), 52*52*3=8112

    h, w = img.shape[:2]         #img 파일의 가로, 세로 크기

    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            # detection: 4(bounding box) + 1(objectness_score) + 80(class confidence)
            scores = detection[5:]     # score에 5~85열 저장, 
            class_id = np.argmax(scores)   #확률값이 가장 높은 열 번호 추출
            confidence = scores[class_id]   # 확률값 저장
            if confidence > confThreshold:
                # 바운딩 박스 중심 좌표 & 박스 크기 구하기
                cx = int(detection[0] * w)  #바운딩 박스 좌표값이 0~1 이므로 이미지 사이즈 곱함
                cy = int(detection[1] * h)
                bw = int(detection[2] * w)
                bh = int(detection[3] * h)

                # 바운딩 박스 좌상단 좌표 구하기
                sx = int(cx - bw / 2)
                sy = int(cy - bh / 2)
                class_ids.append(int(class_id))
                confidences.append(float(confidence))
                boxes.append([sx, sy, bw, bh])
    
    # 비최대 억제; condidences가 가장 높은 boxes 선택
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    for i in indices:    # 이미지에 bounding box와 class, confidence 디스플레이
        sx, sy, bw, bh = boxes[i]
        label = f"{classes[class_ids[i]]}: {confidences[i]:.2}"
        color = colors[class_ids[i]]
        cv2.rectangle(img, (sx, sy, bw, bh), color, 2)
        cv2.putText(img, label, (sx, sy - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)

    cv2.imshow("img", img)  
    cv2.waitKey()
    
cv2.destroyAllWindows()
                